In [12]:
import openai
import os
import pandas as pd
import time
from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain.vectorstores import FAISS
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_community.document_loaders import PyPDFLoader
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from collections import defaultdict
import json
from langchain_text_splitters import RecursiveJsonSplitter, CharacterTextSplitter


load_dotenv()
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
LANGCHAIN_TRACING_V2 = os.getenv('LANGCHAIN_TRACING_V2')
LANGCHAIN_ENDPOINT = os.getenv('LANGCHAIN_ENDPOINT')
LANGCHAIN_API_KEY = os.getenv('LANGCHAIN_API_KEY')
LANGCHAIN_PROJECT = os.getenv('LANGCHAIN_PROJECT')

In [2]:
embedding_function = OpenAIEmbeddings()

def format_docs(docs):
    return "\n\n".join(document.page_content for document in docs)

# Create the language model
llm = ChatOpenAI(model='gpt-4o-mini', temperature=0.0)

analysis_prompt = ChatPromptTemplate.from_messages(
[
    (
    "system",
    """
    You are a professional doctor. Based on the following context, provide a detailed analysis and comprehensive advice regarding the patient's condition. Only include the sections "종합 분석" and "종합적인 조언" in your response. Do not include headings or introductory/concluding sentences. Answer questions using only the provided context. If you do not know the answer, simply state that you do not know; do not speculate or make up information.:\n\n{context}"
     """,
     ),
    ("human", "{question}")
    ]
)

In [3]:
def csv_files_to_json(file_paths, encoding='cp949'):
    # Initialize a default dictionary to store dictionaries for each ID
    json_data = defaultdict(lambda: defaultdict(list))
    
    # Iterate through each CSV file
    for file_path in file_paths:
        # Extract the file name without extension to use as a key
        file_key = file_path.split('/')[-1].split('.')[0]
        
        # Read the CSV file into a DataFrame with cp949 encoding
        df = pd.read_csv(file_path, encoding=encoding)
        
        # Group the data by 'id'
        for id, group in df.groupby('id'):
            # Convert the group data to a dictionary without the 'id' column
            record = group.drop(columns=['id']).to_dict(orient='records')
            # Append the record under the corresponding file name for the given id
            json_data[str(id)][file_key].extend(record)
    
    # Convert the default dictionary to a regular dictionary and then to a JSON string
    json_str = json.dumps(dict(json_data), indent=4)
    
    return json_str

In [4]:
def inbody_analysis(embedding_function, llm, prompt, id, question):
    
    # file_paths = ['./data/inbody.csv', './data/patients.csv','./data/reports.csv','./data/vital.csv']
    file_paths = ['./data/col_value_change/inbody.csv']
    
    inbody_json = csv_files_to_json(file_paths)
    json_data = json.loads(inbody_json)
    
    # 특정 id의 데이터 불러오기 (예: id가 '1'인 경우)
    target_id = id
    
    # 해당 id가 존재하는지 확인
    if target_id in json_data:
        id_data = json_data[target_id]
        print(f"{target_id}에 대한 인바디 정보 분석 중...")
        json_output = json.dumps(id_data)
    else:
        print(f"ID {target_id} not found.")
    
    json_data = json.loads(json_output)
    
    splitter = RecursiveJsonSplitter(max_chunk_size=300)
        
    # The splitter can also output documents
    docs = splitter.create_documents(texts=[json_data])
    
    # or a list of strings
    # texts = splitter.split_text(json_data=json_data)

    #벡터스토어 생성
    inbody_vectorstore = FAISS.from_documents(docs, embedding_function)
    #retriever
    inbody_retriever = inbody_vectorstore.as_retriever()
    inbody_query = f"""
    너는 인바디 정보를 받아 각 환자마다 이를 토대로 추천운동, 추천식단, 건강정보에 중요한 정보들을 정리해야 한다.
    date값은 환자가 측정한 자신의 인바디 측정일이다.
    환자의 인바디 정보가 시간에 따른 변화에서 유의미한 정보가 있다면 정리하는 내용에 포함하라.
    """

    inbody_query += "{context}"
    prompt = ChatPromptTemplate.from_messages(
[
    (
    "system",
    inbody_query,
     ),
    ("human", "{question}")
    ]
)
    
    inbody_chain = (
        {"context": inbody_retriever, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )
    
    inbody_response = inbody_chain.invoke(question)
    return inbody_response

In [5]:
def vital_analysis(embedding_function, llm, prompt, id, question):
    
    # file_paths = ['./data/inbody.csv', './data/patients.csv','./data/reports.csv','./data/vital.csv']
    file_paths = ['./data/col_value_change/vital.csv']
    
    vital_json = csv_files_to_json(file_paths)
    json_data = json.loads(vital_json)
    
    # 특정 id의 데이터 불러오기 (예: id가 '1'인 경우)
    target_id = id
    
    # 해당 id가 존재하는지 확인
    if target_id in json_data:
        id_data = json_data[target_id]
        print(f"{target_id}에 대한 바이탈 정보 분석 중...")
        json_output = json.dumps(id_data)
    else:
        print(f"ID {target_id} not found.")
    
    json_data = json.loads(json_output)
    
    splitter = RecursiveJsonSplitter(max_chunk_size=300)
    
    
    # The splitter can also output documents
    docs = splitter.create_documents(texts=[json_data])
    
    # or a list of strings
    texts = splitter.split_text(json_data=json_data)
    #벡터스토어 생성
    vital_vectorstore = FAISS.from_documents(docs, embedding_function)
    #retriever
    vital_retriever = vital_vectorstore.as_retriever()
    vital_query = f"""
    당신은 환자의 주요 생체 신호 정보를 추천운동, 추천식단, 건강정보에 중요한 정보들을 정리해야 한다. 
    환자의 생체 신호가 시간에 따른 변화에서 유의미한 정보가 있다면 정리하는 내용에 포함하라.
    """

    vital_query += "{context}"
    prompt = ChatPromptTemplate.from_messages(
[
    (
    "system",
    vital_query,
     ),
    ("human", "{question}")
    ]
)
    
    vital_chain = (
        {"context": vital_retriever, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )
    
    vital_response = vital_chain.invoke(question)
    return vital_response

In [6]:
def patients_analysis(embedding_function, llm, id, question):
    
    # file_paths = ['./data/inbody.csv', './data/patients.csv','./data/reports.csv','./data/vital.csv']
    file_paths = ['./data/col_value_change/patients.csv']
    
    patients_json = csv_files_to_json(file_paths)
    json_data = json.loads(patients_json)
    
    # 특정 id의 데이터 불러오기 (예: id가 '1'인 경우)
    target_id = id
    
    # 해당 id가 존재하는지 확인
    if target_id in json_data:
        id_data = json_data[target_id]
        print(f"{target_id}에 대한 환자 정보 분석 중...")
        json_output = json.dumps(id_data)
    else:
        print(f"ID {target_id} not found.")
    
    json_data = json.loads(json_output)
    
    splitter = RecursiveJsonSplitter(max_chunk_size=300)
    
    
    # The splitter can also output documents
    docs = splitter.create_documents(texts=[json_data])
    
    # or a list of strings
    texts = splitter.split_text(json_data=json_data)
    #벡터스토어 생성
    patients_vectorstore = FAISS.from_documents(docs, embedding_function)
    #retriever
    patients_retriever = patients_vectorstore.as_retriever()
    
    patients_query = f"""
    context에 있는 정보를 정리하여 간단하게 출력하라. 현재 날짜는 :{time.strftime('%Y.%m.%d')}이다.
    환자의 나이는 현재 날짜 년도 - 출생년도이다.
    """
    
    patients_query += "{context}"
    prompt = ChatPromptTemplate.from_messages(
[
    (
    "system",
    patients_query,
     ),
    ("human", "{question}")
    ]
)
    
    patients_chain = (
        {"context": patients_retriever, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )
    
    patients_response = patients_chain.invoke(question)
    return patients_response

In [7]:
def reports_analysis(embedding_function, llm, prompt, id, question):
    
    # file_paths = ['./data/inbody.csv', './data/reports.csv','./data/reports.csv','./data/vital.csv']
    file_paths = ['./data/col_value_change/reports.csv']
    
    reports_json = csv_files_to_json(file_paths)
    json_data = json.loads(reports_json)
    
    # 특정 id의 데이터 불러오기 (예: id가 '1'인 경우)
    target_id = id
    
    # 해당 id가 존재하는지 확인
    if target_id in json_data:
        id_data = json_data[target_id]
        print(f"{target_id}에 대한 설문 정보 분석 중...")
        json_output = json.dumps(id_data)
    else:
        print(f"ID {target_id} not found.")
    
    json_data = json.loads(json_output)
    
    splitter = RecursiveJsonSplitter(max_chunk_size=300)
    
    
    # The splitter can also output documents
    docs = splitter.create_documents(texts=[json_data])
    
    # or a list of strings
    texts = splitter.split_text(json_data=json_data)
    #벡터스토어 생성
    reports_vectorstore = FAISS.from_documents(docs, embedding_function)
    #retriever
    reports_retriever = reports_vectorstore.as_retriever()
    
    reports_query = f"""
    당신은 환자가 응답한 설문지에 대한 정보를 추천운동, 추천식단, 건강정보에 중요한 정보들을 정리해야 한다. 
    환자부모에 대한 응답 결과값도 유전력이 있는 질병이라면 이에 대한 점도 유의해야 돼.
    """

    reports_query += "{context}"
    prompt = ChatPromptTemplate.from_messages(
[
    (
    "system",
    reports_query,
     ),
    ("human", "{question}")
    ]
)
    
    reports_chain = (
        {"context": reports_retriever, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )
    
    reports_response = reports_chain.invoke(question)
    return reports_response

In [8]:
def final_response(id, question):
    inbody_response = inbody_analysis(embedding_function, llm, analysis_prompt, id, question)
    vital_response = vital_analysis(embedding_function, llm, analysis_prompt, id, question)
    patients_response = patients_analysis(embedding_function, llm, id, question)
    reports_resonse = reports_analysis(embedding_function, llm, analysis_prompt, id, question)
    total_analysis = f"""
    환자 정보 :
    {patients_response}
    
    inbody 분석 : 
    {inbody_response}
    
    vital 분석 : 
    {vital_response}
    
    설문조사 분석 :
    {reports_resonse} 
    """
    
    final_query = f"""
    You are a professional doctor. Based on the following context, provide a detailed analysis and comprehensive advice regarding the patient's condition. Only include the sections "종합 분석" and "종합적인 조언" in your response. Do not include headings or introductory/concluding sentences. Answer questions using only the provided context. If you do not know the answer, simply state that you do not know; do not speculate or make up information.:\n\n{total_analysis}"""
    
    final_prompt = ChatPromptTemplate.from_messages([
        ("system",
         final_query         
         ),
        ("human", "{question}")
    ])
    final_chain = (
        {"question": RunnablePassthrough()}
        | final_prompt
        | llm
        | StrOutputParser()
    )
    
    response = final_chain.invoke(question)
    return response

In [ ]:
def exercise_reponse(id, question, retriever):
    inbody_response = inbody_analysis(embedding_function, llm, analysis_prompt, id, question)
    vital_response = vital_analysis(embedding_function, llm, analysis_prompt, id, question)
    patients_response = patients_analysis(embedding_function, llm, id, question)
    reports_resonse = reports_analysis(embedding_function, llm, analysis_prompt, id, question)
    total_analysis = f"""
    환자 정보 :
    {patients_response}
    
    inbody 분석 : 
    {inbody_response}
    
    vital 분석 : 
    {vital_response}
    
    설문조사 분석 :
    {reports_resonse} 
    """
    
    final_query = f"""
    As a medical expert, your task is to provide tailored exercise recommendations, including important precautions, based on the patient's medical conditions, fitness level, lifestyle factors, and other relevant health data.

    Using the provided context, your goal is to:

    Analyze the patient's condition comprehensively.
    Recommend appropriate exercises tailored to the patient's needs.
    Highlight important precautions and considerations during exercise to ensure safety and effectiveness.
    Each recommendation and precaution should be supported by clear explanations, referencing clinical guidelines or research when applicable. If there is insufficient information, acknowledge this and suggest a method for the patient to obtain the necessary details, such as consulting with a healthcare provider.

    Do not include personal opinions, speculative advice, or unsupported claims in your responses. Your role is to deliver expert, evidence-based exercise recommendations and safety guidelines tailored to the patient's unique health needs."""
    
    final_prompt = ChatPromptTemplate.from_messages([
        ("system",
         final_query         
         ),
        ("human", "{total_analysis}")
    ])

    final_chain = (
        {
            "context": retriever | RunnableLambda(format_docs),
            "question": RunnablePassthrough(),
        }
        | final_prompt
        | llm
        | StrOutputParser()
    )
    
    response = final_chain.invoke(total_analysis)
    return response

In [ ]:
splitter = CharacterTextSplitter.from_tiktoken_encoder(
        separator="\n",
        chunk_size=1000,
        chunk_overlap=200,
    )
loader = PyPDFLoader("./.cache/files/exercise.pdf")

docs = loader.load_and_split(text_splitter=splitter)
vectorstore = FAISS.from_documents(docs, embedding_function)
retriever = vectorstore.as_retriever()



In [9]:
print(exercise_reponse('7','추천 운동', retriever))

7에 대한 인바디 정보 분석 중...
7에 대한 바이탈 정보 분석 중...
7에 대한 환자 정보 분석 중...
7에 대한 설문 정보 분석 중...
종합 분석: 환자는 47세 남성으로, 최근 체중과 체지방률이 감소하고 근육량이 증가하는 긍정적인 변화를 보이고 있습니다. 체중은 121.6kg에서 118.4kg로 감소하였고, 체지방률도 36.6%에서 35.9%로 줄어들었습니다. 근육량은 증가하여 기초대사량도 함께 증가하는 추세입니다. 혈압은 5월 중순에 급격히 상승했으나 이후 안정세를 보이며 정상 범위로 회복되었습니다. 체온과 산소포화도는 정상 범위에 있어 호흡기 건강이 양호합니다. 그러나 운동 부족이 나타나며, 규칙적인 신체 활동이 필요합니다.

종합적인 조언: 환자는 규칙적인 운동을 통해 심혈관 건강과 체중 관리를 지속해야 합니다. 주 3-5회, 30분 이상의 유산소 운동과 근력 운동을 병행하는 것이 좋습니다. 식단은 저염식, 고섬유질 식단을 유지하며, 과일, 채소, 통곡물, 단백질이 풍부한 음식을 포함해야 합니다. 정기적인 혈압 측정과 체중 관리를 통해 건강 상태를 지속적으로 모니터링하고, 필요시 전문가와 상담하여 개인 맞춤형 건강 관리 계획을 수립하는 것이 중요합니다. 정신적 건강 또한 신체 건강에 영향을 미치므로 스트레스 관리와 정신적 웰빙을 위한 활동을 권장합니다.


In [10]:
print(final_response('7','추천하는 운동을 알려줘'))

7에 대한 인바디 정보 분석 중...
7에 대한 바이탈 정보 분석 중...
7에 대한 환자 정보 분석 중...
7에 대한 설문 정보 분석 중...
종합 분석  
환자는 47세 남성으로, 운동을 하지 않는 상태입니다. 인바디 및 생체 신호 분석에 따르면, 유산소 운동, 근력 운동, 유연성 운동, 균형 운동이 추천됩니다. 특히 걷기, 자전거 타기, 수영과 같은 유산소 운동은 심폐 기능을 향상시키고 체중 조절에 도움을 줄 수 있습니다. 근력 운동으로는 체중을 이용한 운동이나 저항 밴드를 활용한 운동이 적합하며, 이는 근육량 유지와 체중 조절에 기여할 것입니다. 스트레칭과 요가는 유연성을 높이고 부상 예방에 효과적입니다. 

종합적인 조언  
운동을 시작하기 전에 반드시 의사와 상담하여 개인의 건강 상태에 맞는 운동 계획을 세우는 것이 중요합니다. 추천하는 운동은 다음과 같습니다: 매일 30분 이상 빠르게 걷기, 주 2-3회 자전거 타기 또는 수영하기, 주 2-3회 전신 근력 운동 실시, 매일 스트레칭 및 요가를 통해 유연성 향상. 이러한 운동들은 전반적인 건강 증진과 체중 감소에 도움을 줄 것입니다.


In [11]:
print(final_response('7','추천하는 식단들을 설명해'))

7에 대한 인바디 정보 분석 중...
7에 대한 바이탈 정보 분석 중...
7에 대한 환자 정보 분석 중...
7에 대한 설문 정보 분석 중...
저는 요청하신 내용을 제공할 수 없습니다.
